In [1]:
# извлекает аудио из видео
from moviepy.editor import *  

# извлекает текст из аудио (нужен интернет)
import speech_recognition as sr

# нарезает аудио-файл но отрезки по 2 минуты
from pydub import AudioSegment

# прочие либы
import os
from pathlib import Path
import datetime
import shutil

In [2]:
# функция конвертирует mp4 в wav
# и хавает только mp4 файлы

def video2wav(directory_video=str, directory_audio=str, video_file_name=str):
    print("Извлекаем аудиодорожку: " + video_file_name)
    audio = AudioFileClip(directory_video + "/" + video_file_name)
    audio_file_name = video_file_name.replace('.mp4','') + ".wav"
    audio.write_audiofile(directory_audio + "/" + audio_file_name)
    print("Конвертация видео файла закончена!")
    return(audio_file_name)

In [37]:
def start_duration_array(filename_s_d_a=str, inputdir_s_d_a=str):
    audio_seg = AudioSegment.from_wav(inputdir_s_d_a + '//' + filename_s_d_a)
    total_in_ms = len(audio_seg)  # длинна видео в милисек
    
    a = total_in_ms // 120000     # узнаем значение целочисленного деления, для ... 
    b = 120000 * a                # узнаем значение для выхода из цикла
    c = total_in_ms - b           # узнаем значение последнего слогаемого

    s_num = 0
    d_num = 120000
    start_array = [0]           # начальное значение стартового масива
    duration_array = [120000]   # начальное значение другого стартового масива
    
    if total_in_ms > 120000:
        for i in range(a):
            if (b) not in duration_array:  
                s_num = s_num + 120000
                d_num = d_num + 120000
                start_array.append(s_num)
                duration_array.append(d_num)
            else:
                s_num = s_num + 120000
                d_num = d_num + c
                start_array.append(s_num)
                duration_array.append(d_num)      
        dict_start_duration = {'start_array':start_array, 'duration_array':duration_array}
        #print(dict_start_duration)
    else: dict_start_duration = {'start_array':start_array, 'duration_array':duration_array}
        
    return(dict_start_duration)

In [9]:
def create_tmp_aud_files(filename=str, inputdir=str, outputdir=str):
    
    path_tmp_dir = outputdir + '//' + 'wav_' + filename.replace('.wav','')  
    Path(path_tmp_dir).mkdir(parents=True, exist_ok=True)

    myaudio = AudioSegment.from_file(inputdir+"/"+filename, "wav") 

    dict_tmp = start_duration_array(filename_s_d_a=filename, inputdir_s_d_a=inputdir)
    
    for i in range(len(dict_tmp['start_array'])):
        chunk_data = myaudio[dict_tmp['start_array'][i]:dict_tmp['duration_array'][i]]
        chunk_data.export(path_tmp_dir+'/'+str(i)+'_chunk_' + filename, format="wav")
    return(path_tmp_dir)

In [27]:
def wav2txt(tmp_directory_audio=str, tmp_text_path=str, clear_name=str):
    r = sr.Recognizer ()   
    
    # все имена файлов папки сохраняем в список
    _list_name_files = os.listdir(tmp_directory_audio)

    # выводим список имен файлов
    print('\n'.join(_list_name_files))

    for _name_file in _list_name_files:
        #print(type(_name_file))
        #print(_name_file)
        
        name_txt = _name_file.replace("wav", "txt")
        #print(name_txt)
        
        print("Конвертируем в txt файл: \n" + _name_file)

        # извлекаем аудиодорожку
        audio_file = sr.AudioFile(tmp_directory_audio + "/" + _name_file)
        with audio_file as source:
            r.adjust_for_ambient_noise(source) #, duration=0.5) # duration по умолчанию 1 - параметр регулирует шумоподавление
            audio = r.record(source)

        text = r.recognize_google(audio, language = 'ru-RU') 
        
        path_tmp_dir = tmp_text_path + '//' + 'txt_' + clear_name
        #print(path_tmp_dir)
        Path(path_tmp_dir).mkdir(parents=True, exist_ok=True)
        
        with open(path_tmp_dir + '//' + name_txt , "w") as f:
            f.write(text)
        print('!!!Конвертирован: ' + path_tmp_dir + '//' + name_txt)
        
    return(path_tmp_dir)

In [14]:
def txt_combiner(tmp_text_path=str, directory_txt_combine_txt=str):
    text_big = 'start_vidio_text:' + '\n'
    sum_time = datetime.timedelta(seconds=0)
    _list_name_files = os.listdir(tmp_text_path)
    #print('\n'.join(_list_name_files))
    
    for _name_file in _list_name_files: 
        with open(tmp_text_path + '//' + _name_file , "r") as f:
            text = f.read()
            text_big = text_big + str(sum_time) + '\n' + text + '\n'
            sum_time = sum_time + datetime.timedelta(seconds=120)
            
    with open(directory_txt_combine_txt + '//' +'test.txt', "w") as f2:
        f2.write(text_big)   
    print('FINISH')

In [36]:
def clear_tmp_dir(tmp_folder=str):
    _list_name_files = os.listdir(tmp_folder)
    for _name_file in _list_name_files:    
        shutil.rmtree(tmp_folder + '//' + _name_file)
    print('Всременная папка _tmp - очищена')

In [38]:
def main_func(video_file_name=str, tmp_dir_main=str, video_dir_main=str, audio_dir_main=str, txt_dir_main=str):
    
    clear_name_main = video_file_name.replace('.mp4','')

    # конвертируем .mp4 в .wav
    audio_file_name_main = video2wav(directory_video=video_dir_main, directory_audio=audio_dir_main, video_file_name=video_file_name)
    #print(audio_file_name)
    
    # "Нарезаем" аудио файл на отрезки по 120 секунд во временную папку
    tmp_aud_dir_path = create_tmp_aud_files(filename=audio_file_name_main, inputdir=audio_dir_main, outputdir=tmp_dir_main)
    #print(tmp_aud_dir)
    
    # конвертирует .wav в временные .txt
    tmp_txt_dir = wav2txt(tmp_directory_audio=tmp_aud_dir_path, tmp_text_path=tmp_dir_main, clear_name=clear_name_main)
    print(tmp_txt_dir)
    
    # собирает все временные txt и сохранет все один большой txt
    txt_combiner(tmp_text_path=tmp_txt_dir, directory_txt_combine_txt=txt_dir_main)
    
    clear_tmp_dir(tmp_folder=tmp_dir_main)
    
if __name__ == '__main__':
    main_func(video_file_name='interview_test.mp4',
              tmp_dir_main='D://4py//communize_content//_tmp',
              video_dir_main='D://4py//communize_content//video',
              audio_dir_main='D://4py//communize_content//audio',
              txt_dir_main='D://4py//communize_content//txt')

Извлекаем аудиодорожку: interview_test.mp4
MoviePy - Writing audio in D://4py//communize_content//audio/interview_test.wav


MoviePy - Done.
Конвертация видео файла закончена!
0_chunk_interview_test.wav
Конвертируем в txt файл: 
0_chunk_interview_test.wav
!!!Конвертирован: D://4py//communize_content//_tmp//txt_interview_test//0_chunk_interview_test.txt
D://4py//communize_content//_tmp//txt_interview_test
FINISH
Всременная папка _tmp - очищена
